<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#TP-intro-aux-RNNs-en-PyTorch" data-toc-modified-id="TP-intro-aux-RNNs-en-PyTorch-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TP intro aux RNNs en PyTorch</a></span></li><li><span><a href="#Manipulation-basique-de-RNNCell" data-toc-modified-id="Manipulation-basique-de-RNNCell-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Manipulation basique de RNNCell</a></span></li><li><span><a href="#Manipulation-basique-de-RNN" data-toc-modified-id="Manipulation-basique-de-RNN-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Manipulation basique de RNN</a></span></li><li><span><a href="#Données-séquentielles" data-toc-modified-id="Données-séquentielles-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Données séquentielles</a></span></li><li><span><a href="#Limites-des-RNNs-&quot;vanilla&quot;" data-toc-modified-id="Limites-des-RNNs-&quot;vanilla&quot;-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Limites des RNNs "vanilla"</a></span></li><li><span><a href="#RNNs-bidirectionnels" data-toc-modified-id="RNNs-bidirectionnels-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>RNNs bidirectionnels</a></span></li></ul></div>

# TP intro aux RNNs en PyTorch

Lorsque l'on travaille avec des données séquentielles (séries temporelles, phrases, etc.), l'ordre des entrées est crucial pour la tâche à accomplir. Les réseaux neuronaux récurrents (RNN) traitent les données séquentielles en tenant compte de l'entrée courante et de ce qui a été appris des entrées précédentes. Dans ce notebook, nous apprendrons comment encoder des séries temporelles, comment créer et former des RNNs.

![rnn](https://www.irit.fr/~Thomas.Pellegrini/ens/RNN/images/rnn.png)


* **Objectif:** Traiter les données séquentielles en tenant compte de l'entrée courante et de ce qui a été appris des entrées précédentes.
* **Avantages:** 
    * Rendre compte de l'ordre et des entrées précédentes.
    * Génération conditionnée pour générer des séquences.
* **Désavantages:**Désavantages 
    * La prédiction à chaque pas de temps dépend de la prédiction précédente, il est donc difficile de paralléliser les calculs avec un RNN.
    * Le traitement de longues séquences peut entraîner des problèmes de mémoire et de calcul.
    * L'interprétabilité est difficile, mais il y a quelques [techniques](https://arxiv.org/abs/1506.02078) qui utilisent les activations des RNN pour voir quelles parties des entrées sont traitées. 
* **Remarque:** 
    * L'amélioration de l'architecture pour rendre les RNNs plus rapides et interprétables est un domaine de recherche en cours.

![rnn2](https://www.irit.fr/~Thomas.Pellegrini/ens/RNN/images/rnn2.png)

Passe "forward" d'un RNN pour un pas de temps $X_t$ :

$h_t = tanh(W_{hh}h_{t-1} + W_{xh}X_t+b_h)$

$y_t = W_{hy}h_t + b_y $

$ P(y) = softmax(y_t) = \frac{e^y}{\sum e^y} $

*avec*:

* $X_t$ = input au temps t, dans $\mathbb{R}^{NXE}$, avec $N$ la batch size, $E$ la dimension des features (des embeddings si on traite des mots)
* $W_{hh}$ = poids des neurones cachés, dans $\mathbb{R}^{HXH}$, avec $H$ la dim du hidden
* $h_{t-1}$ = état caché au temps précédent, dans $\mathbb{R}^{NXH}$
* $W_{xh}$ = poids sur l'entrée, dans $\mathbb{R}^{EXH}$
* $b_h$ = biais des neurones cachés, dans $\mathbb{R}^{HX1}$
* $W_{hy}$ = poids de la sortie, dans $\mathbb{R}^{HXC}$, avec $C$ le nombre de classes
* $b_y$ = biais des neurones de sortie, dans $\mathbb{R}^{CX1}$

On répète ces calculs pour tous les pas de temps en entrée ($X_{t+1}, X_{t+2}, ..., X_{N})$ pour obtenir une prédiction en sortie à chaque pas de temps.

**Remarque**: Au premier pas de temps, l'état caché précédent $h_{t-1}$ peut être soit un vecteur de zéros ("non-conditionné"), soit initialisé avec certaines valeurs tirées au hasard ou bien fixées par une condition ("conditionné").   

# Manipulation basique de RNNCell

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
batch_size = 5
seq_size = 10 # taille max par input (on utilisera du masking pour les séquences qui sont plus petites que cette valeur)
x_lengths = [8, 5, 4, 10, 5] # taille de chaque séquence en input 
embedding_dim = 100
rnn_hidden_dim = 256
output_dim = 4

In [0]:
# Initialisation des inputs synthétiques
x_in = torch.randn(batch_size, seq_size, embedding_dim)
x_lengths = torch.tensor(x_lengths)
print (x_in.size())

torch.Size([5, 10, 100])


In [0]:
# Initialisation des hidden states (états cachés) à zéro
hidden_t = torch.zeros((batch_size, rnn_hidden_dim))
print (hidden_t.size())

torch.Size([5, 256])


In [0]:
# Initialisation de la cellule RNN
rnn_cell = nn.RNNCell(embedding_dim, rnn_hidden_dim)
print (rnn_cell)

RNNCell(100, 256)


In [0]:
# Passe forward à travers le RNN
x_in = x_in.permute(1, 0, 2) # Le RNN prend la batch_size en dim 1

# On loop sur les pas de temps
hiddens = []
for t in range(seq_size):
    hidden_t = rnn_cell(x_in[t], hidden_t)
    hiddens.append(hidden_t)
hiddens = torch.stack(hiddens)
hiddens = hiddens.permute(1, 0, 2) # on remet la batch_size à la dim 0 (plus logique)
print (hiddens.size())

torch.Size([5, 10, 256])


In [0]:
def gather_last_relevant_hidden(hiddens, x_lengths):
    x_lengths = x_lengths.long().detach().cpu().numpy() - 1
    out = []
    for batch_index, column_index in enumerate(x_lengths):
        out.append(hiddens[batch_index, column_index])
    return torch.stack(out)

In [0]:
# Gather the last relevant hidden state
z = gather_last_relevant_hidden(hiddens, x_lengths)
print (z.size())

torch.Size([5, 256])


In [0]:
# Passe forward dans une couche full-connected 
fc1 = nn.Linear(rnn_hidden_dim, output_dim)
y_pred = fc1(z)
y_pred = F.softmax(y_pred, dim=1)
print (y_pred.size())
print (y_pred)

torch.Size([5, 4])
tensor([[0.2885, 0.2099, 0.2444, 0.2572],
        [0.2612, 0.2535, 0.2398, 0.2455],
        [0.2417, 0.2005, 0.3282, 0.2296],
        [0.2126, 0.2512, 0.2470, 0.2892],
        [0.1783, 0.3209, 0.2918, 0.2090]], grad_fn=<SoftmaxBackward>)


# Manipulation basique de RNN

Nous pouvons utiliser la couche RNN qui est plus haut-niveau que RNNCell (plus abstraite)
pour éviter de faire une boucle (nn.RNN est plus optimisé qu'une boucle)


In [0]:
x_in = torch.randn(batch_size, seq_size, embedding_dim)
rnn = nn.RNN(embedding_dim, rnn_hidden_dim, batch_first=True) # l'option batch_first=True permet de garder la dim batch en premier
out, h_n = rnn(x_in) 
# out : la série temporelle des prédictions
# h_n : le dernier état caché à récupérer pour faire de la classification par exemple

print ("in: ", x_in.size())
print ("out: ", out.size())
print ("h_n: ", h_n.size())

# Y a-t'il une différence entre le dernier vecteur de out et h_n ?
print(out[0, 9, :10] == h_n[0, 0, :10])

in:  torch.Size([5, 10, 100])
out:  torch.Size([5, 10, 256])
h_n:  torch.Size([1, 5, 256])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)


In [0]:
# Passe forward dans une couche full-connected 
fc1 = nn.Linear(rnn_hidden_dim, output_dim)
y_pred = fc1(h_n)
y_pred = F.softmax(y_pred, dim=1)
print (y_pred.size())
print (y_pred)

torch.Size([1, 5, 4])
tensor([[[0.2527, 0.1788, 0.1901, 0.2795],
         [0.1751, 0.2291, 0.1916, 0.2059],
         [0.2314, 0.1812, 0.2669, 0.1534],
         [0.1756, 0.2396, 0.1576, 0.1979],
         [0.1652, 0.1713, 0.1938, 0.1633]]], grad_fn=<SoftmaxBackward>)


# Données séquentielles

Plusieurs types de tâches séquentielles peuvent être réalisées par des RNNs.

1. **One-to-one** : une entrée génère une sortie. 
    * Ex. Donner un mot et prédire sa catégorie syntaxique (verbe, nom, etc.).
    
2. **One-to-Many** : une entrée génère plusieurs sorties.
    * Ex. Prédire une opinion (positive, négative, etc., on parle de sentiment analysis), générer une critique.

3. **Many-to-one** : de nombreuses entrées sont traitées séquentiellement pour générer une seule sortie.
    * Ex. Traiter les mots dans une critique pour prédire sa "valence" (positive ou négative).

4. **Many-to-many** : de nombreuses entrées sont traitées séquentiellement pour générer de nombreuses sorties.
    * Ex. Le modèle encode une phrase en français, il traite toute la phrase, puis en produit la traduction anglaise.
    * Ex. Étant donnée une série de données chronologiques, prédire la probabilité d'un événement (risque de maladie) à chaque temps.

![seq2seq](https://www.irit.fr/~Thomas.Pellegrini/ens/RNN/images/seq2seq.jpeg)

# Limites des RNNs "vanilla"

Il y a plusieurs problèmes avec les RNN simples (dits "vanilla" en anglais) que nous avons vus ci-dessus. 

1. Lorsque nous avons une très longue séquence d'entrée, il devient difficile pour le modèle de conserver l'information vue plus tôt à mesure que nous traitons la séquence. L'objectif du modèle est de conserver les informations utiles des pas de temps précédents, mais cela devient impossible pour une taille de séquence trop grande.

2. Pendant la rétropropropagation, le gradient de la fonction de perte doit remonter jusqu'au premier pas de temps. Si notre gradient est supérieur à 1 (${1.01}^{1000} = 20959$) ou inférieur à 1 (${{0.99}^{1000} = 4.31e-5$) et que nous avons beaucoup de pas de temps, cela peut rapidement dégénérer.

Pour répondre à ces deux questions, le concept de "porte" ("gate") a été introduit dans les RNN. Les gates permettent aux RNN de contrôler le flux d'information entre les étapes temporelles afin d'optimiser la tâche à réaliser. Le fait de laisser passer sélectivement l'information permet au modèle de traiter des données séquentielles très longues. Les variantes les plus courantes des RNN sont les unités de mémoire à court terme, appelées [LSTM](https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM), et les unités récurrentes à "porte" [GRU](https://pytorch.org/docs/stable/nn.html#torch.nn.GRU). Vous pouvez en savoir plus sur le fonctionnement de ces unités [ici](http://colah.github.io/posts/2015-08-Understanding-LSTMs/).


![rnn](https://www.irit.fr/~Thomas.Pellegrini/ens/RNN/images/gates.png)

In [0]:
# GRU in PyTorch
gru = nn.GRU(input_size=embedding_dim, hidden_size=rnn_hidden_dim, 
             batch_first=True)

In [0]:
# Initialize synthetic input
x_in = torch.randn(batch_size, seq_size, embedding_dim)
print (x_in.size())

torch.Size([5, 10, 100])


In [0]:
# Forward pass
out, h_n = gru(x_in)
print ("out:", out.size())
print ("h_n:", h_n.size())

out: torch.Size([5, 10, 256])
h_n: torch.Size([1, 5, 256])


In [0]:
x = torch.randn(5, 3, 7)
gru = nn.GRU(7, 20)
out = gru(x)
print(out[0].size(), out[1].size())


torch.Size([5, 3, 20]) torch.Size([1, 3, 20])


**Remarque**: Le choix d'utiliser des GRU ou des LSTM dépend des données et des performances empiriques. Les GRU offrent des performances comparables avec un nombre réduit de paramètres, tandis que les LSTM sont plus efficaces et peuvent faire la différence en termes de performances pour une tâche particulière.

# RNNs bidirectionnels

Beaucoup de progrès ont été réalisés ces dernières années avec les RNN, comme par exemple l'introduction de mécanismes d'[attention](https://www.oreilly.com/ideas/interpretability-via-attentional-and-memory-based-interfaces-using-tensorflow), les Quasi-RNNs, etc. L'une de ces avancées, largement utilisée, sont les RNNNs bidirectionnels (Bi-RNNs). La motivation derrière les RNN bidirectionnels est de traiter une séquence d'entrée dans les deux sens. La prise en compte du contexte dans les deux sens peut améliorer la performance lorsque toute la séquence d'entrée est connue au moment de l'inférence. Une application courante des Bi-RNNs est la traduction automatique : il est avantageux de considérer une phrase entière dans les deux sens pour la traduire dans une autre langue.


![rnn](https://www.irit.fr/~Thomas.Pellegrini/ens/RNN/images/birnn.png)

In [0]:
# BiGRU en PyTorch
bi_gru = nn.GRU(input_size=embedding_dim, hidden_size=rnn_hidden_dim, 
                batch_first=True, bidirectional=True)

In [0]:
# Passe forward 
out, h_n = bi_gru(x_in)
print ("out:", out.size()) # tenseur contenant tous les hidden states du RNN
print ("h_n:", h_n.size()) # le dernier hidden state du RNN

out: torch.Size([5, 10, 512])
h_n: torch.Size([2, 5, 256])


La sortie à chaque temps a une taille de 512, le double de la dim cachée précisée lors de la création de la couche GRU. Cela s'explique par le fait qu'elle inclut à la fois les directions avant et arrière encodées par le BiRNNN. 

https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79


# Gérer les séquences de taille variable

In [0]:
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable

## Solution 1 : zero-padding

Pour construire un tensor qui regroupe des séquences de taille différente, on complète les séquences avec des zéros jusqu'à obtenir la taille de la séquence la plus grande.


Cela est réalisé à l'aide de pad_sequence.

In [5]:
# sur des séquences 1-d
a = torch.Tensor([1, 2, 3])
b = torch.Tensor([4, 5])
c = torch.Tensor([6])
rnn_utils.pad_sequence([a, b, c], batch_first=False, padding_value=0) # ce sont les arguments par défaut.

tensor([[1., 4., 6.],
        [2., 5., 0.],
        [3., 0., 0.]])

In [6]:
# sur des séquences 2-d (ici des "embeddings")
embedding_dim = 100
lengths = [10, 15, 20]
a = torch.randn(lengths[0], embedding_dim)
b = torch.randn(lengths[1], embedding_dim)
c = torch.randn(lengths[2], embedding_dim)

x_in = rnn_utils.pad_sequence([a, b, c])
print (x_in.size())


torch.Size([20, 3, 100])


On a obtenu un tenseur de dimensions : T x B x d

La fonction pad_sequence suppose que les dimensions des séquences, autres que leur taille, sont identiques. D'autre part, les tenseurs doivent être de même type.


## Solution 2 : "empaqueter" les séquences avec pack_sequence

In [7]:
# sur des séquences 1-d
a = torch.Tensor([1, 2, 3])
b = torch.Tensor([4, 5])
c = torch.Tensor([6])
# rnn_utils.pack_sequence([a, b, c], enforce_sorted=True) # enforce_sorted: vérifie que les sequences sont déjà ordenées par longueur décroissante
rnn_utils.pack_sequence([a, b, c])

PackedSequence(data=tensor([1., 4., 6., 2., 5., 3.]), batch_sizes=tensor([3, 2, 1]), sorted_indices=None, unsorted_indices=None)

In [8]:
# sur des séquences 2-d (ici des "embeddings")
embedding_dim = 100
lengths = [10, 15, 20]
# lengths = lengths[::-1]
a = torch.randn(lengths[0], embedding_dim)
b = torch.randn(lengths[1], embedding_dim)
c = torch.randn(lengths[2], embedding_dim)

# x_in = rnn_utils.pack_sequence([a, b, c], enforce_sorted=False)
# x_in = rnn_utils.pack_sequence([a, b, c])
# --> RuntimeError: 'lengths' array has to be sorted in decreasing order
x_in = rnn_utils.pack_sequence([c, b, a])

print (x_in.data.size())
print(x_in.batch_sizes)
print(len(x_in.batch_sizes))
print(x_in.batch_sizes.sum())

torch.Size([45, 100])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1])
20
tensor(45)


## Solution 3 : pack_padded_sequence

In [9]:
# sur des séquences 2-d (ici des "embeddings")
embedding_dim = 100
lengths = [10, 15, 20]
lengths = lengths[::-1]
a = torch.randn(lengths[0], embedding_dim)
b = torch.randn(lengths[1], embedding_dim)
c = torch.randn(lengths[2], embedding_dim)

x_pad = rnn_utils.pad_sequence([a, b, c])
print (x_pad.size())

# x_in = rnn_utils.pack_padded_sequence(x_pad, torch.Tensor(lengths),  enforce_sorted=False)
x_in = rnn_utils.pack_padded_sequence(x_pad, torch.Tensor(lengths))
print(x_in.data.size(), x_in.batch_sizes)

torch.Size([20, 3, 100])
torch.Size([45, 100]) tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1])


## Application : mini-batch de sequences de tailles variables dans un RNN

In [10]:
# sur des séquences 2-d (ici des "embeddings")
embedding_dim = 100
lengths = [10, 15, 20]
lengths = lengths[::-1]
a = torch.randn(lengths[0], embedding_dim)
b = torch.randn(lengths[1], embedding_dim)
c = torch.randn(lengths[2], embedding_dim)

x_pad = rnn_utils.pad_sequence([a, b, c], batch_first=False)
print (x_pad.size())

# x_in = rnn_utils.pack_padded_sequence(x_pad, torch.Tensor(lengths),  enforce_sorted=False)
x_in = rnn_utils.pack_padded_sequence(x_pad, torch.Tensor(lengths), batch_first=False)
print(x_in.data.size(), x_in.batch_sizes)

torch.Size([20, 3, 100])
torch.Size([45, 100]) tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1])


In [11]:
# now run through LSTM
num_layers=1
hidden_size=10
batch_size=3

rnn = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=False,
        )

h0 = Variable(torch.randn(num_layers, batch_size, hidden_size))
c0 = Variable(torch.randn(num_layers, batch_size, hidden_size))

out, (h, c) = rnn(x_in, (h0, c0))
# 
# h: hidden state for t = seq_len 
# c: cell state for t = seq_len
print(h.size(), c.size()) 
print(type(out))

torch.Size([1, 3, 10]) torch.Size([1, 3, 10])
<class 'torch.nn.utils.rnn.PackedSequence'>


out est un objet PackedSequence. On peut défaire le packing avec pad_packed_sequence :

In [12]:
# undo the packing operation
out_unpacked, _ = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=False)
out_unpacked.size()

torch.Size([20, 3, 10])

In [13]:
# Pour passer out à une couche linéaire, on fait un reshape, avec un contiguous avant pour
# accélérer les accès mémoire : 
out_vec = out_unpacked.contiguous()
out_vec = out_vec.view(-1, out_vec.shape[2])
out_vec.size()

torch.Size([60, 10])

out_vec est maintenant une matrice de size (T*B, H).

On peut appliquer maintenant une couche linéaire.

In [14]:
num_classes = 5
classif_layer = nn.Linear(hidden_size, num_classes)
classif_out = classif_layer(out_vec)
classif_out.size()

torch.Size([60, 5])

Nous pouvons ffaire un reshape pour récupérer des séquences de prédictions

In [15]:
preds = classif_out.reshape(-1, batch_size, num_classes)
print(preds.size())

torch.Size([20, 3, 5])
